In [1]:
# Necessary Imports

from tqdm import tqdm_notebook
import pathlib
import matplotlib.animation as animation
import imageio
import os
import numpy as np
import matplotlib.pyplot as plt
# 
def save_fig(fig_id,path, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(path, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [4]:
# This Function Gets the Folders of the Loadcases and creates the Modified Data Folder
# and also gets the Path for that

def loadcase_folder_paths(DATA_PATH, MODIFIED_DATA_PATH):
    folder_paths = []
    folder_names = [name for name in os.listdir(DATA_PATH)]
    data_root= pathlib.Path(DATA_PATH)
    for item in data_root.iterdir():
        folder_paths.append(str(item))
        os.makedirs(item, exist_ok=True)
    modified_folder_paths= []
    for name in folder_names:
        NEW_FOLDER_PATH = os.path.join(MODIFIED_DATA_PATH,str(name))
        modified_folder_paths.append(NEW_FOLDER_PATH)
        os.makedirs(NEW_FOLDER_PATH, exist_ok=True)

    return folder_paths, modified_folder_paths
# FOLDER_PATHS, MODIFIED_FOLDER_PATHS = loadcase_folder_paths(DATA_PATH,MODIFIED_DATA_PATH)
# print(FOLDER_PATHS, MODIFIED_FOLDER_PATHS)

In [5]:
# This gets all the individual Paths of the Loadcases for all the Folders

from pathlib import Path
def get_data_names_paths(FOLDER_PATHS):
    DATA_NAMES = []
    DATA_PATHS = []
    for folder in FOLDER_PATHS:
        FOLDER_i_names= [name for name in os.listdir(folder)]
        FOLDER_i_paths=[]       
        folder = Path(folder)
        for item in folder.iterdir():
            FOLDER_i_paths.append(item)
        DATA_NAMES.append(FOLDER_i_names)
        DATA_PATHS.append(FOLDER_i_paths)
    return np.array(DATA_NAMES),np.array(DATA_PATHS)
            
# names, paths = get_data_names_paths(FOLDER_PATHS)

In [6]:
# Read in the files as a list of the individual lines of the files
def line_by_line(path_):
  as_list=[]
  f = open(path_, 'r')
  for line in f:      
      as_list.append(line.strip())
  return as_list


# OUT DATA
# Get the number of nodes in the Structure
def get_num_nodes(list_):
  numnodes_i= [i for i, s in enumerate(list_) if 'Total # of Grids (Structural)' in s]
  step1=list_[numnodes_i[0]].split()
  num_nodes=(int(step1[-1]))
  return num_nodes

# Get the number of elements in the structure
def get_num_elements(list_):
  numelements_i= [i for i, s in enumerate(list_) if 'Total # of Elements' in s]
  step1=list_[numelements_i[0]].split()
  num_elements=(int(step1[-1]))
  return num_elements


# Input Deck Data
# DON'T FORGET!: We start indexing at 0, so Node 132 for example is node_pos[131]

# Extracting the volume fraction
def get_vol_frac(list_):
  volfrac_i= [i for i, s in enumerate(list_) if 'DCONSTR  ' in s]
  volfrac_i= int(volfrac_i[0])
  step1=list_[volfrac_i].split()
  volfrac=(float(step1[-1]))
  return volfrac

# We get the position of nodes, in this case the element size is 2.5
def get_nodes_pos(list_,num_nodes):
# Defining the area, in which the nodes positions are saved (grid_area)
  grid_start_i= [i for i, s in enumerate(list_) if '  GRID Data' in s]
  pos= []
  grid_start_i=int(grid_start_i[0])+2
  grid_end_i = int(grid_start_i+int(num_nodes))
  grid_area= list_[grid_start_i:grid_end_i]
# Now we are accessing each line in the grid area and getting the nodes positions, 
# those have the string length of 8 characters, thats why we are accessing them through indexes
# Also there are some cases in which the position is not 0, but some very small number like:
# 3.553-15120, this is why we check also for "-" in the position
  for i,s in enumerate(grid_area):
  
    x_pos= s[24:32]
    y_pos= s[32:40]
    if "-" in x_pos:
      x_pos= 0
    else:
      x_pos= int(float(x_pos)/2.5)
    if "-" in y_pos:
      y_pos= 0
    else:
      y_pos= int(float(y_pos)/2.5)   
    pos.append([x_pos,y_pos])  
  return pos

def get_elements_pos(list_,num_elements,nodes_pos):
# Our Elements pos is equal to the node position of the lower left node
# Since we read in the Nodes mentioned in the Input deck, we have to substract 1 from
# the Node defined in the input deck, since we start indexing at 0
# Define the area containing the elements
  ele_start_i= [i for i, s in enumerate(list_) if '  CQUAD4 Elements' in s]
  pos= []
  ele_start_i=int(ele_start_i[0])+2
  ele_end_i = int(ele_start_i+int(num_elements))
  ele_area= list_[ele_start_i:ele_end_i]
  for i,s in enumerate(ele_area):
# Split the Nodes, which define the Element
    s= s.split()
# We only need the lower left node, which is the second node defined in the QUAD
# We substract -1 to get the right node element in our vector, since indexing
# This approach didnt work out, and is therefore commented out
    # node_2 = int(s[4])-1
    # node_2_pos=nodes_pos[node_2]
    # x_pos=node_2_pos[0]
    # y_pos=node_2_pos[1]
    in_1=int(s[3])-1
    in_2=int(s[4])-1
    in_3=int(s[5])-1
    in_4=int(s[6])-1
# The Approach with the second node doesnt work for some reason
# Thats why we just take the minimal entry for the lower left node
    min_pos= (min(nodes_pos[in_1],nodes_pos[in_2],nodes_pos[in_3],nodes_pos[in_4]))
    x_pos=min_pos[0]
    y_pos=min_pos[1]
    pos.append([x_pos,y_pos])
    
  return pos

# Takes the nodes positions and volfrac and makes a volfrac and node matrix
# The Volfrac matrix is just a matrix filled with the volume fraction
# The Node matrix is a matrix containing the sorted nodes positions
def get_nodes_volfrac_matrix(nodes_pos,volfrac):
# Shape doesn't care about indexing, since the 0th entry counts as well
  max_x =max(i[0] for i in nodes_pos)+1
  max_y =max(i[1] for i in nodes_pos)+1
  node_matrix= np.zeros((max_x,max_y))
  for i,s in enumerate(nodes_pos):
    i+=1
    x_=s[0]
    y_=s[1]
    node_matrix[x_,y_]=i

  volfrac_matrix=np.full((max_x-1,max_y-1),volfrac)
  return node_matrix.astype(int), volfrac_matrix.astype(float),max_x,max_y

# Same with the element matrix
def get_elements_matrix(elements_pos):
  max_x =max(i[0] for i in elements_pos)+1
  max_y =max(i[1] for i in elements_pos)+1 
  elements_matrix= np.zeros((max_x,max_y))
  for i,s in enumerate(elements_pos):  
    i+=1
    x_=s[0]
    y_=s[1]
    elements_matrix[x_,y_]=i
  return elements_matrix.astype(int),max_x,max_y

def get_volfrac_matrix(prototype,volfrac):
  volfrac_matrix = np.full_like(prototype,float(volfrac))
  return volfrac_matrix


# Displacement Data
# No need to adjust the nodes number by adding +1, since the Displacements also start indexing with 0

def get_displacement_matrixes(nodes_pos, displacements,num_nodes,x_shape,y_shape):
  iter_array_i= [i for i, s in enumerate(displacements) if "ITER   " in s]
# We only need the first iteration 
  start = iter_array_i[0]+2
  end = start+num_nodes
  iter_1= displacements[start:end]
# Make empty float arrays
  matrix_x=np.zeros((x_shape,y_shape))
  matrix_y=np.zeros((x_shape,y_shape))
  for i,s in enumerate(iter_1):
    s=str(s)
# Get the corresponding position of the node with index i
    node_pos=nodes_pos[i]
# Split into x and y coordinates
    node_x= node_pos[0]
    node_y= node_pos[1]
# Come from the back, since the space doesn't match up, because of grid numbers
# Change to float and get it in the right position on the matrix
    x_disp=float(s[-72:-60])
    y_disp=float(s[-60:-48])
    matrix_x[node_x,node_y]=x_disp
    matrix_y[node_x,node_y]=y_disp
  return matrix_x,matrix_y

# Strain Data
def get_strain_matrix(elements_pos, strains,num_elements,x_shape,y_shape):
  iter_array_i= [i for i, s in enumerate(strains) if "ITER   " in s]
# Basically same as before only with 3 Matrixes
  start = iter_array_i[0]+2
  end = start+num_elements
  iter_1= strains[start:end]
# Make empty float arrays
  matrix_x=np.zeros((x_shape,y_shape))
  matrix_y=np.zeros((x_shape,y_shape)) 
  matrix_xy=np.zeros((x_shape,y_shape))  

  for i,s in enumerate(iter_1):
    s=str(s)
    # print(s)
    element_pos=elements_pos[i]
    elements_x= element_pos[0]
    elements_y= element_pos[1]
    x_strains=s[-60:-48]
    y_strains=s[-36:-24]
    xy_strains=s[-12:]
    matrix_x[elements_x,elements_y]=float(x_strains)
    matrix_y[elements_x,elements_y]=float(y_strains)
    matrix_xy[elements_x,elements_y]=float(xy_strains)
  return matrix_x,matrix_y, matrix_xy  

# Saves ALL iterations of the Structural optimization
def get_structures(list_,elements_pos,num_elements,x_shape,y_shape):
  iter_array_i= [i for i, s in enumerate(list_) if "iter " in s]
  matrixes=[]
  for i in iter_array_i:
    curr_iter_start= int(i)+1
    curr_iter_end = curr_iter_start+num_elements
    matrix=np.zeros((x_shape,y_shape))
    curr_iter= list_[curr_iter_start:curr_iter_end]
    for i,s in enumerate(curr_iter):
      pos= elements_pos[i]
      x_pos= pos[0]
      y_pos= pos[1]
      s=str(s)
      density = s[-12:]
      matrix[x_pos,y_pos]= float(density)
    matrixes.append(matrix)

  return np.array(matrixes)


# NEW: FORCES
# from the change.fem we will extract the forces and sort them in x and y to the corresponding 
# place in the node matrix
def get_forces(list_):
  iter_array_i_change= [i for i, s in enumerate(list_) if "FORCE " in s]
  start_force = iter_array_i_change[0]
  area = list_[start_force:]
  force_nodes= []
  force_x = []
  force_y = []
  for i in area:
    i_spl = i.split()
#  WE START INDEXING THE NODES AT 0, so we have to substract 1
    force_nodes.append(int(i_spl[2])-1)
    force_x.append(float(i[40:48]))
    force_y.append(float(i[48:56]))
  return np.array(force_nodes),np.array(force_x),np.array(force_y)
    
def get_forces_matrixes(nodes_pos,force_nodes,force_x,force_y,x_shape_nodes,y_shape_nodes):
  force_nodes_pos=[]
  for i in force_nodes:

#     pos = nodes_pos[i-1]
    pos = nodes_pos[i]
    force_nodes_pos.append(pos)
  x_force_matrix = np.zeros((x_shape_nodes,y_shape_nodes))
  y_force_matrix = np.zeros((x_shape_nodes,y_shape_nodes))
  by_nodes = []
  for i,s in enumerate(force_nodes):
    by_nodes.append([s,force_x[i],force_y[i]])
    
  for i,s in enumerate(force_nodes_pos):
    x_ = int(s[0])
    y_ = int(s[1])
    x_force_matrix[x_,y_]= force_x[i]
    y_force_matrix[x_,y_]= force_y[i]
  return np.array(x_force_matrix),np.array(y_force_matrix),np.array(by_nodes)

In [7]:
# Use all defined functions to create the matrixes

def create_instance(folder_name,DATA_PATH):
  final=[]
  DATA_PATH = os.path.join(DATA_PATH, folder_name)
  paths=[]
  path_names= ['inputDeck.fem','inputDeck.disp','inputDeck.dens', 'inputDeck.out','inputDeck.strs','change.fem']
  for path_name in path_names:
    path_ = os.path.join(DATA_PATH, path_name)
    paths.append(path_)
  data_0 = []  
  for path_ in paths:
    data_read= line_by_line(path_)
    data_0.append(data_read)
# Out
  num_nodes = get_num_nodes(data_0[3])
  num_elements = get_num_elements(data_0[3])
# fem
  volfrac = get_vol_frac(data_0[5])
  nodes_pos = get_nodes_pos(data_0[0],num_nodes)
  elements_pos = get_elements_pos(data_0[0],num_elements,nodes_pos)
  nodes_matrix,volfrac_matrix,x_shape_nodes,y_shape_nodes = get_nodes_volfrac_matrix(nodes_pos,volfrac)
# NEW: change
  force_nodes,force_x,force_y = get_forces(data_0[5])
  x_force_matrix, y_force_matrix,by_nodes = get_forces_matrixes(nodes_pos,force_nodes,force_x,force_y,x_shape_nodes,y_shape_nodes)  
# First final Matrix is ready - Volfrac is index 0
  final.append(np.array(np.rot90(volfrac_matrix)))
  elements_matrix, x_shape_elements,y_shape_elements = get_elements_matrix(elements_pos)
# Displacement Data
  x_disp,y_disp=get_displacement_matrixes(nodes_pos, data_0[1], num_nodes, x_shape_nodes, y_shape_nodes)
# Second final Matrix - X- Dipslacements - index is 1
  final.append(np.array(np.rot90(x_disp)))
# Third final Matrix  - Y - Displacements - index is 2
  final.append(np.array(np.rot90(y_disp)))
# Strain Matrix Data
  x_strains,y_strains,xy_strains= get_strain_matrix(elements_pos, data_0[4],num_elements,x_shape_elements,y_shape_elements)
# Last three final Matrixes are ready, X,Y, XY Strain with indexes 3,4,5
  final.append(np.array(np.rot90(x_strains)))
  final.append(np.array(np.rot90(y_strains)))
  final.append(np.array(np.rot90(xy_strains)))
  return final


In [ ]:
# WE ADRESS NODES AND ELEMENTS AT 0, SINCE WE START INDEXING AT 0; SO
# IF YOU NEED TO ADRESS A SPECIFIC NODE -1!!!!!
def modifiy_data(folder_path,data_names,save_root):
    faulty= []
    checkpoint_path = os.path.join(save_root,"checkpoint.npy")
    try:
        checkpoint = np.load(checkpoint_path)
        checkpoint = int(checkpoint)-3

        print(checkpoint)
    except:
        checkpoint = 0
        print(checkpoint)
    for i,s in tqdm_notebook(enumerate(data_names[checkpoint:])):
        checkpoint = i

        np.save(checkpoint_path,checkpoint)
        try:
            modified_path= os.path.join(save_root,s+".npy")
    #         print(s)
    #         print(modified_path)
            a = create_instance(str(s),folder_path)
            np.save(modified_path, a)
        except:
            faulty.append(s)
    faulty_path = os.path.join(save_root,"faulty.npy")
    np.save(faulty_path,faulty)
    return faulty
        
# print(FOLDER_PATHS)
# faulty = modifiy_data(FOLDER_PATHS[0],names[0],MODIFIED_FOLDER_PATHS[0])

In [2]:
def full_auto(DATA_PATH, MODIFIED_DATA_PATH):
    FOLDER_PATHS, MODIFIED_FOLDER_PATHS = loadcase_folder_paths(DATA_PATH,MODIFIED_DATA_PATH)
    names, paths = get_data_names_paths(FOLDER_PATHS)
    for i,s in enumerate(FOLDER_PATHS):
        print(FOLDER_PATHS[i])
        faulty = modifiy_data(FOLDER_PATHS[i],names[i],MODIFIED_FOLDER_PATHS[i])
        
full_auto(DATA_PATH,MODIFIED_DATA_PATH)
    
    

NameError: name 'DATA_PATH' is not defined